In [11]:
import gym
import numpy as np
env = gym.make('CartPole-v1', render_mode="rgb_array")
observation = env.reset()

print(observation)

(array([0.0474103 , 0.00768469, 0.03536078, 0.00068417], dtype=float32), {})


In [12]:
"""
observation = state
reward = reward
done = whether it's time to reset environment
"""

env.reset()
for k in range(100):
    env.render()
    print(observation)
    action = env.action_space.sample()
    observation, reward, done, _, info = env.step(action)
    if done:
        print("Episode finished after {} time steps".format(k + 1))
        break

env.close()

(array([0.0474103 , 0.00768469, 0.03536078, 0.00068417], dtype=float32), {})
[ 0.0462951   0.14573042 -0.00352884 -0.29547256]
[ 0.04920971 -0.04934104 -0.00943829 -0.00390467]
[ 0.04822289 -0.24432637 -0.00951638  0.2857855 ]
[ 0.04333636 -0.43931133 -0.00380067  0.5754519 ]
[ 0.03455014 -0.24413629  0.00770837  0.28157407]
[ 0.02966741 -0.43936735  0.01333985  0.57667816]
[ 0.02088006 -0.2444349   0.02487341  0.28822732]
[ 0.01599137 -0.43990254  0.03063796  0.58865017]
[ 0.00719331 -0.24522273  0.04241096  0.30577362]
[ 0.00228886 -0.05073     0.04852644  0.02676194]
[ 0.00127426  0.14366366  0.04906167 -0.25022414]
[ 0.00414753 -0.05212333  0.04405719  0.05752131]
[ 0.00310507 -0.24784838  0.04520762  0.36377263]
[-0.0018519  -0.05339712  0.05248307  0.08568025]
[-0.00291984 -0.24923056  0.05419667  0.39444888]
[-0.00790446 -0.445078    0.06208565  0.70371467]
[-0.01680601 -0.6410029   0.07615995  1.0152773 ]
[-0.02962607 -0.8370533   0.09646549  1.3308698 ]
[-0.04636714 -0.6432713

In [13]:
n_bins_pos = 10
n_bins_vel = 10
n_bins_ang = 10
n_bins_anv = 10

def map_discrete_state(state):
    pos, vel, ang, anv = state

    idx_pos = np.where(np.histogram(np.clip(pos, -2, 2), bins= n_bins_pos, range=(-2, 2))[0] == 1)[0][0]
    idx_vel = np.where(np.histogram(np.clip(vel, -2, 2), bins= n_bins_vel, range=(-2, 2))[0] == 1)[0][0]
    idx_ang = np.where(np.histogram(np.clip(ang, -0.4, 0.4), bins= n_bins_ang, range=(-0.4, 0.4))[0] == 1)[0][0]
    idx_anv = np.where(np.histogram(np.clip(anv, -2, 2), bins= n_bins_anv, range=(-2, 2))[0] == 1)[0][0]

    states = np.zeros([n_bins_pos, n_bins_vel, n_bins_ang, n_bins_anv])
    states[idx_pos, idx_vel, idx_ang, idx_ang] = 1

    states = states.reshape(-1, 1)
    s = np.where(states == 1)[0][0]

    return s

In [14]:
n_states = n_bins_pos * n_bins_vel * n_bins_ang * n_bins_anv
n_actions = 2


In [15]:
alpha = 0.3
gamma = 0.9

Q_table = np.random.uniform(0, 1, (n_states, n_actions))

for episode in range(801):
    done = False
    state = env.reset()
    state = tuple(state[0])

    while not done:
        s = map_discrete_state(state)

        epsilon = 0.1
        if np.random.uniform() < epsilon:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q_table[s, :])

        next_state, reward, done, _, _ = env.step(a)

        if done: 
            reward = -100
            Q_table[s, a] = reward
        else:
            next_s = map_discrete_state(next_state)
            Q_table[s, a] = (1 - alpha) * Q_table[s, a] + alpha * reward + gamma * np.max(Q_table[next_s, :])
        state = next_state

env.close()

/private/tmp/PKInstallSandbox.oNjx7s/tmp/ipykernel_62979/2226680874.py:27: RuntimeWarning: overflow encountered in scalar add
  Q_table[s, a] = (1 - alpha) * Q_table[s, a] + alpha * reward + gamma * np.max(Q_table[next_s, :])


In [16]:
Q_table

array([[0.35606817, 0.55556244],
       [0.26443476, 0.54034142],
       [0.87293781, 0.32369064],
       ...,
       [0.01518989, 0.84195374],
       [0.4994129 , 0.10684662],
       [0.96481926, 0.89767094]])

In [17]:
state = env.reset()
state = tuple(state[0])
done = False
while not done:
    env.render()
    s = map_discrete_state(state)
    action = np.argmax(Q_table[s, :])
    next_state, _, done, _, _ = env.step(action)
    state = next_state

env.close()
